In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
dataset = pd.read_csv('/content/train_data.txt', sep=':::', names=['Movie_title', 'Genre', 'Description'], engine='python')

In [ ]:
dataset.head()

,Movie_title,Genre,Description
1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
2,Cupid (1997),thriller,A brother and sister with a past incestuous r...
3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...


In [ ]:
dataset.shape

(54214, 3)

In [ ]:
dataset.isnull().sum()

,0
Movie_title,0
Genre,0
Description,0


In [ ]:
dataset.describe()

,Movie_title,Genre,Description
count,54214,54214,54214
unique,54214,27,54086
top,Nature's Fury: Storm of the Century (2006),drama,Grammy - music award of the American academy ...
freq,1,13613,12


In [ ]:
dataset['Genre'].unique()

array([' drama ', ' thriller ', ' adult ', ' documentary ', ' comedy ',
       ' crime ', ' reality-tv ', ' horror ', ' sport ', ' animation ',
       ' action ', ' fantasy ', ' short ', ' sci-fi ', ' music ',
       ' adventure ', ' talk-show ', ' western ', ' family ', ' mystery ',
       ' history ', ' news ', ' biography ', ' romance ', ' game-show ',
       ' musical ', ' war '], dtype=object)

In [ ]:
dataset.duplicated().sum()

0

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')


corpus =[]

for i in range(0, len(dataset)):
  # '[^a-zA-Z]' means leave all letter form a to z or A to Z and replace any thing else with a space [' ']
  review = re.sub('[^a-zA-Z]', ' ', dataset.iloc[i]['Description'])
  # Lower the letter
  review = review.lower()
  #tokinizing or splitting words
  review = review.split()
  #ps = PorterStemmer()
  lemmatizer = WordNetLemmatizer()
  all_stopwords = stopwords.words('english')
  # apply stemming on the reviews except stop words
  #review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = [lemmatizer.lemmatize(word) for word in review if not word in set(all_stopwords)]
  # joining the reviews thogether again with space between them [' '].
  review = ' '.join(review)
  # add the cleaned reviews into the corpus list
  corpus.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
corpus[0]

'listening conversation doctor parent year old oscar learns nobody courage tell week live furious refuse speak anyone except straight talking rose lady pink meet hospital stair christmas approach rose us fantastical experience professional wrestler imagination wit charm allow oscar live life love full company friend pop corn einstein bacon childhood sweetheart peggy blue'

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X = tf.fit_transform(corpus).toarray()

In [ ]:
y = dataset.iloc[:, 1]

In [ ]:
y

,Genre
1,drama
2,thriller
3,adult
4,drama
5,drama
...,...
54210,comedy
54211,horror
54212,documentary
54213,comedy


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

MultinomialNB()

In [ ]:
from sklearn.metrics import accuracy_score
y_pred_n = classifier.predict(X_test)
print(accuracy_score(y_test, y_pred_n))

0.5248547449967721


In [ ]:
from sklearn.linear_model import LogisticRegression
classifier_l = LogisticRegression()
classifier_l.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
y_pred_l = classifier_l.predict(X_test)
print(accuracy_score(y_test, y_pred_l))

0.5932859909619109


In [ ]:
from sklearn.model_selection import cross_val_score
# cv -> num of folds
#another parameter -->[n_jops = -1] if i have a big dataset (it makes the cpu works with all effort)
accuracies = cross_val_score(estimator= classifier_l, X = X_train, y = y_train, cv = 10)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [ ]:
print(accuracies.mean())

0.5823013147827995


In [ ]:
new_pred = 'When Earth becomes uninhabitable in the future, a farmer and ex-NASA pilot, Joseph Cooper, is tasked to pilot a spacecraft, along with a team of researchers, to find a new planet for humans.'

new_pred = re.sub('[^a-zA-Z]', ' ', new_pred)
  # Lower the letter
new_pred= new_pred.lower()
  #tokinizing or splitting words
new_pred = new_pred.split()
  #ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()
all_stopwords = stopwords.words('english')
  # apply stemming on the reviews except stop words
  #review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
new_pred = [lemmatizer.lemmatize(word) for word in new_pred if not word in set(all_stopwords)]
  # joining the reviews thogether again with space between them [' '].
new_pred = ' '.join(new_pred)
tf_new = tf.transform([new_pred]).toarray()
print(classifier_l.predict(tf_new))

[20]
